In [4]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)


Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:705)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:749)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:673)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:639)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:615)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:572)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:530)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [148]:
tbl_merchants = spark.read.parquet("../data/tables/tbl_merchants.parquet")

In [149]:
from pyspark.sql.functions import *
tbl_merchants = tbl_merchants.toPandas()

In [150]:
# Observe strings format
tbl_merchants["tags"][0]

'((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))'

In [155]:
# Separate tags from revenue and take rate
tags1 = tbl_merchants["tags"].str.split("\), ", expand=True)
tags2 = tags[0].str.split("\], ", expand=True)

In [156]:
# Remove symbols from tag and making everything lowercase
tags3 = tags2[0].str.replace('[^\w\s]', '', regex = True)
tags4 = tags3.str.lower()

In [157]:
tags4

0       furniture home furnishings and equipment shops...
1       cable satellite and other pay television and r...
2                jewelry watch clock and silverware shops
3                    watch clock and jewelry repair shops
4       music shops  musical instruments pianos and sh...
                              ...                        
4021               opticians optical goods and eyeglasses
4022                     books periodicals and newspapers
4023                                          shoe  shops
4024                 motor vehicle supplies and new parts
4025                 motor vehicle supplies and new parts
Name: 0, Length: 4026, dtype: object

In [139]:
# https://relatedwords.io/furniture
furniture = ["house", "table", "room", "wood", "floor", "chair", "cabinet", \
             "items", "goods", "oak", "paintings", "renaissance", "bed", \
             "clothing", "kitchen", "clothes", "furniture", "floors", "baroque", \
             "decorations", "carpet", "toys", "pottery", "americana", "accessories", \
             "jewelry", "textiles", "lamp", "article of furniture", "neolithic", \
             "couch", "tile", "floor coverings", "appliances", "cabinets", \
             "antiques", "furnishings", "fabrics", "decor", "decorating", "carpets", \
             "flooring", "bedding", "furnishing", "mattress", "woodworking", "rugs", \
             "upholstery", "glassware", "wood flooring"]

In [140]:
# https://relatedwords.io/electronics
electronics = ["games", "game", "video", "radio", "technology", "parts", "digital", \
               "entertainment", "internet", "car", "engineering", "security", "computer", \
               "metal", "electric", "equipment", "programming", "photo", "electronic", \
               "audio", "software", "motor", "mobile", "tech", "physics", "device", "phone", \
               "mechanical", "electricals", "lights", "electrical", "devices", "electronics", \
               "electricity", "battery", "computers", "hardware", "mechanics", "clothes", \
               "furniture", "gaming", "automotive", "micro", "wireless", "aerospace", \
               "automated", "toys", "digital cameras", "batteries", "stereos"]

In [141]:
# https://relatedwords.io/fashion
fashion = ["style", "clothing", "outfit", "lifestyle", "beauty", "vogue", "haute couture", \
           "trend", "tailor", "manner", "couture", "designer", "styles", "dresses", "retro", \
           "mode", "makeup", "cut", "footwear", "designers", "fashion", "lingerie", "way", \
           "fit", "make", "clothes", "art", "design", "forge", "menswear", "stylist", \
           "fashion designer", "women wear", "form", "aesthetic", "pattern", "society", \
           "lanvin", "music", "fashionistas", "catwalks", "editorial", "language", "brand", \
           "versace", "dress", "michael kors", "paris fashion week", "fashionably" ,"vivienne westwood"]

In [145]:
# https://relatedwords.io/beauty-products
beauty = ["perfume", "beauty", "hair", "lipstick", "rouge", "beautician", \
          "food and drug administration", "cream", "lotion", "fine", "grace", \
          "lovely", "product", "art", "mascara", "scent", "hairspray", "shampoo"]
# https://relatedwords.io/health
health = ["family", "improve", "education", "research", "social", "health", \
          "families", "medical", "food", "hospital", "care", "medicine", "agriculture", "safety"]
# https://relatedwords.io/household
household = ["people", "family", "life", "living", "personal", "work", "home", \
             "house", "community", "business", "person", "close", "domestic", \
             "child", "individual", "food", "parents", "car"]

beauty_health_personal_household = beauty + health + household

In [146]:
# https://relatedwords.io/hobby
hobby = ["sports", "art", "game", "sport", "interest", "amateur", "photography", "fun", \
         "recreation", "profession", "hunting", "passion", "craft", "pursuit", "lifestyle", \
         "recreational", "genealogy", "leisure"]
# https://relatedwords.io/toys
toys = ["play", "outfits", "doll", "lego", "plastic", "meccano", "dally", "dolls", \
        "ball", "candy", "miniature", "yo-yo", "barbie", "hobby", "teddy bear", "baby", "dollhouse"]
# https://relatedwords.io/diy
DIY = ["repair", "art", "ikea", "gardening", "mod", "do it yourself", "repairing", \
       "tlc", "repairs", "soho", "trug", "duluth", "cooke", "faq", "adirondack"]

toys_hobby_DIY = hobby + toys + DIY

In [168]:
# https://www.shopify.com/au/blog/top-online-shopping-categories
categories = [fashion, furniture, electronics, beauty_health_personal_household, toys_hobby_DIY]
categories_label = ["fashion", "furniture", "electronics", "beauty, health, personal and household", "toys, hobbies and DIY"]

In [183]:
import builtins as p
for i in range(0, len(tags4)):
    counts = [0]*len(categories)
    for j in range(0, len(categories)):
        for word in categories[j]:
            if word in tags4[i]:
                counts[j] += 1
    if counts == [0]*len(categories):
        tags4[i] = "other"
    else:   
        max_value = p.max(counts)
        max_index = counts.index(max_value)
        tags4[i] = categories_label[max_index]

In [184]:
tags4

0         furniture
1       electronics
2         furniture
3         furniture
4           fashion
           ...     
4021      furniture
4022          other
4023          other
4024    electronics
4025    electronics
Name: 0, Length: 4026, dtype: object

In [186]:
type(tags4)

pandas.core.series.Series

In [187]:
tbl_merchants["categoriesed_tags"] = tags4

In [190]:
tbl_merchants.head(20)

,name,tags,merchant_abn,categoriesed_tags
0,Felis Limited,"((furniture, home furnishings and equipment sh...",10023283211,furniture
1,Arcu Ac Orci Corporation,"([cable, satellite, and otHer pay television a...",10142254217,electronics
2,Nunc Sed Company,"([jewelry, watch, clock, and silverware shops]...",10165489824,furniture
3,Ultricies Dignissim Lacus Foundation,"([wAtch, clock, and jewelry repair shops], [b]...",10187291046,furniture
4,Enim Condimentum PC,"([music shops - musical instruments, pianos, a...",10192359162,fashion
5,Fusce Company,"[(gift, card, novelty, and souvenir shops), (a...",10206519221,electronics
6,Aliquam Enim Incorporated,"[(computers, comPUter peripheral equipment, an...",10255988167,electronics
7,Ipsum Primis Ltd,"[[watch, clock, and jewelry repair shops], [c]...",10264435225,furniture
8,Pede Ultrices Industries,"([computer programming , data processing, and ...",10279061213,electronics
9,Nunc Inc.,"[(furniture, home furnishings and equipment sh...",10323485998,furniture
